## Analyze the kaggle code and extract to code blocks with step name, columns, methods and reasons

In [169]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [170]:
import numpy as np
import pandas as pd
import json

from config import upload_folder, allowed_extensions, open_api_mode, jupyter_url, openai_key
from utils import read_code, response, update_prompt, prompt_list_len, read_codev1, read_output_from_nb, write_code

# show unlimited pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)

# show unlimited numpy
np.set_printoptions(threshold=np.inf)

# do not show backtrace in python error
import sys
sys.tracebacklimit = 0

In [171]:
def print_output(output):
    import warnings
    warnings.filterwarnings("ignore")
    import sys
    from io import StringIO
    from contextlib import redirect_stdout
    
    code = 'print("".join({}))'.format(output)
    f = StringIO()
    with redirect_stdout(f):
        exec(code)
        out_value = f.getvalue()
    
    return out_value

def print_output2(output):
    import warnings
    warnings.filterwarnings("ignore")
    import sys
    from io import StringIO
    from contextlib import redirect_stdout
    
    code = 'print(f"{}")'.format(output)
    print(code)
    f = StringIO()
    with redirect_stdout(f):
        exec(code)
        out_value = f.getvalue()
    
    return out_value

In [195]:
import psycopg2 as pydb
class Database:
    def __init__(self):
        host = "localhost"
        port = 5432
        user = "musicadmin"
        password = "gsds2023#"
        dbname = "promptdb"
        self.con = pydb.connect(host=host, port=port, user=user, password=password, database=dbname)
        self.cur = self.con.cursor()

    def close(self):
        self.cur.close()
        self.con.close()

    """
    for code table
    """
    def list_codes(self):
        self.cur.execute("SELECT c.*, k.ref AS kernel_ref, d.ref AS dataset_ref FROM codes as c INNER JOIN kernels as k ON c.kernel_id = k.id INNER JOIN datasets d ON k.dataset_id = d.id ORDER BY c.id")
        result = self.cur.fetchall()
        self.close()
        return result
    
    def list_codes_by_kernel(self, kernel_id):
        self.cur.execute("SELECT c.*, k.ref AS kernel_ref, d.ref AS dataset_ref FROM codes as c INNER JOIN kernels as k ON c.kernel_id = k.id INNER JOIN datasets d ON k.dataset_id = d.id WHERE c.kernel_id=%s ORDER BY c.id", (kernel_id,))
        result = self.cur.fetchall()
        self.close()
        return result
    
    def list_kernels_from_codes(self):
        self.cur.execute("SELECT DISTINCT k.id, k.ref FROM codes as c INNER JOIN kernels as k ON c.kernel_id = k.id ORDER BY k.id")
        result = self.cur.fetchall()
        self.close()
        return result
    
    """
    for workflow table
    """
    def list_workflow(self):
        self.cur.execute("SELECT k.ref AS kernel, w.* FROM workflow as w INNER JOIN kernels as k ON w.kernel_id = k.id ORDER BY w.step")
        result = self.cur.fetchall()
        self.close()
        return result
    
    def list_workflow_by_kernel(self, kernel_id):
        self.cur.execute("SELECT k.ref AS kernel, w.* FROM workflow as w INNER JOIN kernels as k ON w.kernel_id = k.id WHERE w.kernel_id=%s ORDER BY w.step", (kernel_id,))
        result = self.cur.fetchall()
        self.close()
        return result
    
    def insert_workflow(self, kernel_id, workflow):
        # check kernel_id
        self.cur.execute("SELECT id FROM kernels WHERE id = %s", (kernel_id,))
        result = self.cur.fetchall()
        if result and len(result) > 0:
            # build data with kernel_id
            data = []
            for task in workflow:
                step = task['Step']
                task_name = task['Task Name']
                column_names = task['Column Names']
                method = task['Method']
                reason = task['Reason']
                data.append((kernel_id, step, task_name, column_names, method, reason))
            # insert code
            self.cur.executemany("INSERT INTO workflow(kernel_id, step, task_name, column_names, method, reason) VALUES (%s, %s, %s, %s, %s, %s)", data)
            self.con.commit()
            self.close()
            return self.cur.rowcount
        else:
            print('Kernel {} does not exist!'.format(kernel_id))
            return 0

    # insert generation by batch
    def insert_gen(self, kernel_id, gen_data):
        # check kernel_id
        self.cur.execute("SELECT id FROM kernels WHERE id = %s", (kernel_id,))
        result = self.cur.fetchall()
        if result and len(result) > 0:
            version = 0
            # build data with kernel_id
            data = []
            for d in gen_data:
                step = d['step']
                prompt = d['prompt']
                code = d['code']
                output = d['output']
                data.append((kernel_id, version, step, prompt, code, output))
            # insert code
            self.cur.executemany("INSERT INTO generation(kernel_id, version, step, prompt, code, output) VALUES (%s, %s, %s, %s, %s, %s)", data)
            self.con.commit()
            self.close()
            return self.cur.rowcount
        else:
            print('Kernel {} does not exist!'.format(kernel_id))
            return 0


In [225]:
def generate_steps(problem, code):
    # create prompt for chatgpt
    prompt_content = 'From the problem description, summarize the given code in the following json format: step {name, column names, strategy, reason}.'
    code += '\n---------------\n'
    if len(problem) > 0:
        problem = 'Problem Description: ' + problem + '\n'
    prompt = code + problem + prompt_content
    print(prompt)

    # call openai
    out = response(openai_key, prompt, temperature=0.7)
    print('>>', out)
    return out

In [174]:
db = Database()
codes = db.list_codes()
len(codes)

3278

In [226]:
db = Database()
kernel_id = 51
codes = db.list_codes_by_kernel(kernel_id)
print(len(codes))

problem = 'House price prediction in India.'
all_steps = []
begin_idx = 15
batch = 15
end_idx = min(begin_idx + batch, len(codes))
while begin_idx < len(codes):
    part_codes = codes[begin_idx:end_idx]
    code = [c[2] + '\n' for c in part_codes]
    code = print_output(code)

    steps = generate_steps(problem, code)
    all_steps.append(steps)
    
    begin_idx = end_idx
    end_idx = min(begin_idx + batch, len(codes))
    

28
sns.barplot(data=df1,x='number of bathrooms',y='Price')

plt.figure(figsize=(12,18))
sns.countplot(data=df1,y='Built Year')
plt.show()

plt.figure(figsize=(12,18))
sns.countplot(data=df1,y='Postal Code')
plt.show()

df1=df1.drop(['id'],axis=1)

df1=df1.drop(['Postal Code'],axis=1)

plt.figure(figsize=(15,15))
sns.heatmap(df1.corr(),linewidths=0.5,annot=True,cmap='Blues')
plt.show()

df1=df1.drop(['lot area','condition of the house','Built Year','lot_area_renov','Number of schools nearby','Distance from the airport','Longitude'],axis=1)

from sklearn.model_selection import train_test_split 
X=df1.drop(['Price'],axis =1)
y=df1['Price']

X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=11)

pipelines = {
    'en':make_pipeline(StandardScaler(), ElasticNet()),
    'lasso':make_pipeline(StandardScaler(), Lasso()),
    'Rcv':make_pipeline(StandardScaler(), RidgeCV()),
    'CatB':make_pipeline(StandardScaler(), CatBoostRegressor(eval_metric='RMSE',verbose=1000

In [227]:
import json
#step_json = json.loads('[' + all_steps[0] + ']')
step_json = json.loads(all_steps[0])
step_json

{'step1': {'name': 'Data Visualization',
  'column names': ['number of bathrooms', 'Price'],
  'strategy': 'Bar plot',
  'reason': 'To visualize the relationship between the number of bathrooms and the price of houses.'},
 'step2': {'name': 'Data Visualization',
  'column names': ['Built Year'],
  'strategy': 'Count plot',
  'reason': 'To visualize the distribution of houses based on their built year.'},
 'step3': {'name': 'Data Visualization',
  'column names': ['Postal Code'],
  'strategy': 'Count plot',
  'reason': 'To visualize the distribution of houses based on their postal code.'},
 'step4': {'name': 'Data Preprocessing',
  'column names': ['id'],
  'strategy': 'Drop column',
  'reason': "To remove the 'id' column which is not relevant for the house price prediction."},
 'step5': {'name': 'Data Preprocessing',
  'column names': ['Postal Code'],
  'strategy': 'Drop column',
  'reason': "To remove the 'Postal Code' column which is not relevant for the house price prediction."},
 '

In [232]:
import re
workflow = []
target_column = 'Price'
pass_keys = ''
stepidx = 10
for k in step_json.keys():
    if k in pass_keys:
        continue
    step = step_json[k]
    stepidx += 1
    task_name = step['Name'.lower()]
    if step.get('Column_Names'.lower()):
        column_names = step['Column_Names'.lower()]
    else:
        column_names = step['Column Names'.lower()]
    if isinstance(column_names, list):
        column_names = ','.join(column_names)
    else:
        column_names = column_names.replace(' and ', ',')
        column_names = column_names.replace("'", "").replace('"','')
    #print(column_names)
    # check task name
    pattern = r"(load|read)"
    if re.search(pattern, task_name, re.IGNORECASE):
        column_names = 'All'
    #pattern = r"(train|test|validate|evaluation|evaluate)"
    #if re.search(pattern, task_name, re.IGNORECASE):
    #    column_names = target_column
        
    if 'All' in column_names:
        column_names = 'All'
    elif len(column_names) == 0:
        column_names = 'All'
    elif 'N/A' in column_names:
        column_names = 'All'
    method = step['Strategy'.lower()]
    reason = step['Reason'.lower()]
    
    task = {'Step': stepidx, 'Task Name': task_name, 'Column Names': column_names, 'Method': method, 'Reason': reason}
    workflow.append(task)
workflow

[{'Step': 11,
  'Task Name': 'Data Visualization',
  'Column Names': 'number of bathrooms,Price',
  'Method': 'Bar plot',
  'Reason': 'To visualize the relationship between the number of bathrooms and the price of houses.'},
 {'Step': 12,
  'Task Name': 'Data Visualization',
  'Column Names': 'Built Year',
  'Method': 'Count plot',
  'Reason': 'To visualize the distribution of houses based on their built year.'},
 {'Step': 13,
  'Task Name': 'Data Visualization',
  'Column Names': 'Postal Code',
  'Method': 'Count plot',
  'Reason': 'To visualize the distribution of houses based on their postal code.'},
 {'Step': 14,
  'Task Name': 'Data Preprocessing',
  'Column Names': 'id',
  'Method': 'Drop column',
  'Reason': "To remove the 'id' column which is not relevant for the house price prediction."},
 {'Step': 15,
  'Task Name': 'Data Preprocessing',
  'Column Names': 'Postal Code',
  'Method': 'Drop column',
  'Reason': "To remove the 'Postal Code' column which is not relevant for the ho

In [231]:
# insert workflow
rowcnt = 0
db = Database()
rowcnt += db.insert_workflow(kernel_id, workflow)
print('Inserted {} row(s)'.format(rowcnt))

Inserted 12 row(s)


In [236]:
from utils import print_str

# read prompt, code, output from nb_file
prompts = []
codes = []
outputs = []

nb_file = "../kaggle/kernels_train/abrish2003/house-rent-eda-along-with-price-prediction/house-rent-eda-along-with-price-prediction.ipynb"

nb_code = json.load(open(nb_file))
nb_cells = nb_code['cells']
for cell_idx in range(len(nb_cells)):
    cell = nb_cells[cell_idx]
    if cell['cell_type'] == 'markdown' and '-' in cell['source'][0]:
        prompt = cell['source'][0].replace('-', '').strip()
        code_cell = nb_cells[cell_idx + 1]
        if code_cell['cell_type'] == 'code':
            code = code_cell['source']
            code = print_str(code)
            output = ''
            for out in code_cell['outputs']:
                if out.get('name') == 'stdout' and out.get('output_type') == 'stream':
                    output = out['text']
                    output = print_str(output)
                    break
            prompts.append(prompt)
            codes.append(code)
            outputs.append(output)

In [ ]:
outputs

In [235]:
db = Database()
gen_data = []
for i in range(len(prompts)):
    gen_data.append({'step':i, 'prompt':prompts[i], 'code': codes[i], 'output': outputs[i]})
db.insert_gen(kernel_id, gen_data)

35